In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import random
from sklearn.preprocessing import normalize

In [2]:
def cut(contents, cut=2):
    results = []
    for content in contents:
        words = content.split()
        result = []
        for word in words:
            result.append(word[:cut])
        results.append(' '.join([token for token in result]))
    return results

In [3]:
def divide(x, y, train_prop):
    random.seed(1234)
    x = np.array(x)
    y = np.array(y)
    tmp = np.random.permutation(np.arange(len(x)))
    x_tr = x[tmp][:round(train_prop * len(x))]
    y_tr = y[tmp][:round(train_prop * len(x))]
    x_te = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_te = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return x_tr, x_te, y_tr, y_te

In [4]:
def make_input(documents, max_document_length):
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_length)  # 객체 선언
    x = np.array(list(vocab_processor.fit_transform(documents)))
    vocab_dict = vocab_processor.vocabulary_._mapping
    sorted_vocab = sorted(vocab_dict.items(), key=lambda x: x[1])
    vocabulary = list(list(zip(*sorted_vocab))[0])
    return x, vocabulary, len(vocab_processor.vocabulary_)

In [5]:
def make_output(points, threshold):
    results = np.zeros((len(points),2))
    for idx, point in enumerate(points):
        if point > threshold:
            results[idx,0] = 1
        else:
            results[idx,1] = 1
    return results

In [6]:
def check_maxlength(contents):
    max_document_length = 0
    for document in contents:
        document_length = len(document.split())
        if document_length > max_document_length:
            max_document_length = document_length
    return max_document_length

In [7]:
def loading_rdata(data_path):
    corpus = pd.read_table(data_path, sep=",", encoding="utf-8")
    corpus = np.array(corpus)
    contents = []
    points = []
    for i in range(len(corpus)):
        contents.append(corpus[i][0])
        points.append(corpus[i][1])
    return contents, points

In [8]:
def isNumber(s):
  try:
    float(s)
    return True
  except ValueError:
    return False

In [9]:
import os
import time
import datetime
from tensorflow import flags
import tensorflow as tf
import numpy as np
import cnn_tool as tool

In [10]:
class TextCNN(object):
    def __init__(
            self, sequence_length, num_classes, vocab_size,
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        """
        <Variable>
            - W: 각 단어의 임베디드 벡터의 성분을 랜덤하게 할당
        """
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
        #with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(labels = self.scores, logits = self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [11]:
# data loading
data_path = '/Users/justin/Desktop/Justin/2018-1/Capstone/Code/Practice2.csv'
contents, points = tool.loading_rdata(data_path)
contents = tool.cut(contents,cut=2)

In [12]:
len(contents)

129734

In [14]:
# tranform document to vector
max_document_length = 200
x, vocabulary, vocab_size = tool.make_input(contents,max_document_length)
print('사전단어수 : %s' % (vocab_size))
y = tool.make_output(points,threshold=2.5)


사전단어수 : 52320


In [15]:
type(vocabulary)

list

In [16]:
# divide dataset into train/test set
x_train, x_test, y_train, y_test = tool.divide(x,y,train_prop=0.8)

In [17]:
y_train[1:2]

array([[1., 0.]])

In [18]:
# Model Hyperparameters
flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of embedded vector (default: 128)")
flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

In [19]:
# Training parameters
flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [20]:
# Misc Parameters
flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [26]:
FLAGS = tf.flags.FLAGS
FLAGS.__flags.items()

dict_items([('embedding_dim', <absl.flags._flag.Flag object at 0x1c261303c8>), ('filter_sizes', <absl.flags._flag.Flag object at 0x1c26130940>), ('num_filters', <absl.flags._flag.Flag object at 0x1c26130c50>), ('dropout_keep_prob', <absl.flags._flag.Flag object at 0x1c261308d0>), ('l2_reg_lambda', <absl.flags._flag.Flag object at 0x1c261309b0>), ('batch_size', <absl.flags._flag.Flag object at 0x1c261307f0>), ('num_epochs', <absl.flags._flag.Flag object at 0x1c26130a20>), ('evaluate_every', <absl.flags._flag.Flag object at 0x1c26130ac8>), ('checkpoint_every', <absl.flags._flag.Flag object at 0x1c26130710>), ('num_checkpoints', <absl.flags._flag.Flag object at 0x1c261302b0>), ('allow_soft_placement', <absl.flags._flag.BooleanFlag object at 0x1c25676cf8>), ('log_device_placement', <absl.flags._flag.BooleanFlag object at 0x1c25676dd8>), ('h', <tensorflow.python.platform.app._HelpFlag object at 0x1c25676d68>), ('help', <tensorflow.python.platform.app._HelpFlag object at 0x1c25676d68>), ('he

In [27]:
FLAGS = tf.flags.FLAGS
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x1c25676cf8>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x1c261307f0>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x1c26130710>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x1c261308d0>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x1c261303c8>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x1c26130ac8>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x1c26130940>
H=<tensorflow.python.platform.app._HelpFlag object at 0x1c25676d68>
HELP=<tensorflow.python.platform.app._HelpFlag object at 0x1c25676d68>
HELPFULL=<tensorflow.python.platform.app._HelpfullFlag object at 0x1a1ef19588>
HELPSHORT=<tensorflow.python.platform.app._HelpshortFlag object at 0x1c25676f60>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x1c261309b0>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x1c25676dd8>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x1c261302b0>
NUM_EPOCHS=<absl.flags._flag.Flag object at 

In [28]:
FLAGS.l2_reg_lambda

0.1

In [29]:
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN(sequence_length=x_train.shape[1],
                      num_classes=y_train.shape[1],
                      vocab_size=vocab_size,
                      embedding_size=FLAGS.embedding_dim,
                      filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                      num_filters=FLAGS.num_filters,
                      l2_reg_lambda=FLAGS.l2_reg_lambda)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)


        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        def batch_iter(data, batch_size, num_epochs, shuffle=True):
            """
            Generates a batch iterator for a dataset.
            """
            data = np.array(data)
            data_size = len(data)
            num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
            for epoch in range(num_epochs):
                # Shuffle the data at each epoch
                if shuffle:
                    shuffle_indices = np.random.permutation(np.arange(data_size))
                    shuffled_data = data[shuffle_indices]
                else:
                    shuffled_data = data
                for batch_num in range(num_batches_per_epoch):
                    start_index = batch_num * batch_size
                    end_index = min((batch_num + 1) * batch_size, data_size)
                    yield shuffled_data[start_index:end_index]


        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

        testpoint = 0
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                if testpoint + 100 < len(x_test):
                    testpoint += 100
                else:
                    testpoint = 0
                print("\nEvaluation:")
                dev_step(x_test[testpoint:testpoint+100], y_test[testpoint:testpoint+100], writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Summary name W:0 is illegal; using W_0 instead.
INFO:tensorflow:Summary name W:0 is illegal; using W_0 instead.
INFO:tensorflow:Summary name output/b:0 is illegal; using output/b_0 instead.
INFO:tensorflow:Summary name output/b:0 is illegal; using output/b_0 instead.
Writing to /Users/justin/Desktop/Justin/2018-1/Capstone/Code/runs/1528109774

2018-06-04T19:56:14.690445: step 1, loss 2.24779, acc 0.765625
2018-06-04T19:56:14.763305: step 2, loss 2.65747, acc 0.75
2018-06-04T19:56:14.832541: step 3, loss 2.27936, acc 0.734375
2018-06-04T19:56:14.904986: step 4, loss 2.75745, acc 0.75
2018-06-04T19:56:14.993967: step 5, loss 2.55912, acc 0.734375
2018-06-04T19:56:15.063149: step 6, loss 2.09846, acc 0.78125
2018-06-04T19:56:15.137650: step 7, loss 2.61088, acc 0.671875
2018-06-04T19:

2018-06-04T19:56:24.934729: step 118, loss 0.3769, acc 0.5
2018-06-04T19:56:25.017663: step 119, loss 0.374275, acc 0.578125
2018-06-04T19:56:25.112284: step 120, loss 0.451439, acc 0.46875
2018-06-04T19:56:25.223295: step 121, loss 0.472496, acc 0.625
2018-06-04T19:56:25.301146: step 122, loss 0.466334, acc 0.484375
2018-06-04T19:56:25.372184: step 123, loss 0.634448, acc 0.46875
2018-06-04T19:56:25.443978: step 124, loss 0.42147, acc 0.53125
2018-06-04T19:56:25.520853: step 125, loss 0.297803, acc 0.46875
2018-06-04T19:56:25.598799: step 126, loss 0.352765, acc 0.5625
2018-06-04T19:56:25.673374: step 127, loss 0.415666, acc 0.375
2018-06-04T19:56:25.769570: step 128, loss 0.435194, acc 0.4375
2018-06-04T19:56:25.860343: step 129, loss 0.373967, acc 0.390625
2018-06-04T19:56:25.955830: step 130, loss 0.472787, acc 0.53125
2018-06-04T19:56:26.034475: step 131, loss 0.394586, acc 0.578125
2018-06-04T19:56:26.152329: step 132, loss 0.358304, acc 0.53125
2018-06-04T19:56:26.278519: step 1

2018-06-04T19:56:35.655450: step 241, loss 0.0238823, acc 0.4375
2018-06-04T19:56:35.746735: step 242, loss 0.0256168, acc 0.359375
2018-06-04T19:56:35.817626: step 243, loss 0.0254499, acc 0.328125
2018-06-04T19:56:35.899032: step 244, loss 0.034966, acc 0.328125
2018-06-04T19:56:35.978290: step 245, loss 0.0301561, acc 0.359375
2018-06-04T19:56:36.053410: step 246, loss 0.021673, acc 0.328125
2018-06-04T19:56:36.132404: step 247, loss 0.0192617, acc 0.5
2018-06-04T19:56:36.205899: step 248, loss 0.0275783, acc 0.484375
2018-06-04T19:56:36.278698: step 249, loss 0.0237572, acc 0.3125
2018-06-04T19:56:36.352332: step 250, loss 0.0173287, acc 0.53125
2018-06-04T19:56:36.430033: step 251, loss 0.0223484, acc 0.46875
2018-06-04T19:56:36.498541: step 252, loss 0.018942, acc 0.421875
2018-06-04T19:56:36.571527: step 253, loss 0.0202757, acc 0.40625
2018-06-04T19:56:36.677320: step 254, loss 0.0156209, acc 0.390625
2018-06-04T19:56:36.798640: step 255, loss 0.0167224, acc 0.40625
2018-06-04T

2018-06-04T19:56:45.449576: step 363, loss 0.000760755, acc 0.46875
2018-06-04T19:56:45.530095: step 364, loss 0.000830824, acc 0.359375
2018-06-04T19:56:45.602280: step 365, loss 0.000610469, acc 0.53125
2018-06-04T19:56:45.679772: step 366, loss 0.000757148, acc 0.390625
2018-06-04T19:56:45.770738: step 367, loss 0.000687174, acc 0.484375
2018-06-04T19:56:45.850357: step 368, loss 0.000711524, acc 0.46875
2018-06-04T19:56:45.936061: step 369, loss 0.00057026, acc 0.453125
2018-06-04T19:56:46.007843: step 370, loss 0.000530758, acc 0.5
2018-06-04T19:56:46.079390: step 371, loss 0.000535342, acc 0.484375
2018-06-04T19:56:46.196520: step 372, loss 0.000556433, acc 0.546875
2018-06-04T19:56:46.296117: step 373, loss 0.000608455, acc 0.4375
2018-06-04T19:56:46.378302: step 374, loss 0.000370877, acc 0.5
2018-06-04T19:56:46.459420: step 375, loss 0.00051063, acc 0.46875
2018-06-04T19:56:46.530920: step 376, loss 0.000492704, acc 0.390625
2018-06-04T19:56:46.600090: step 377, loss 0.0005545

2018-06-04T19:56:57.530464: step 483, loss 2.69201e-06, acc 0.671875
2018-06-04T19:56:57.638853: step 484, loss 3.34733e-06, acc 0.59375
2018-06-04T19:56:57.724847: step 485, loss 3.35263e-06, acc 0.65625
2018-06-04T19:56:57.818522: step 486, loss 2.84479e-06, acc 0.75
2018-06-04T19:56:57.903558: step 487, loss 2.40022e-06, acc 0.6875
2018-06-04T19:56:58.013222: step 488, loss 2.60147e-06, acc 0.640625
2018-06-04T19:56:58.119104: step 489, loss 2.41447e-06, acc 0.796875
2018-06-04T19:56:58.216148: step 490, loss 2.32205e-06, acc 0.65625
2018-06-04T19:56:58.301998: step 491, loss 2.02986e-06, acc 0.6875
2018-06-04T19:56:58.413850: step 492, loss 2.10934e-06, acc 0.703125
2018-06-04T19:56:58.508286: step 493, loss 2.1887e-06, acc 0.6875
2018-06-04T19:56:58.581252: step 494, loss 1.87159e-06, acc 0.640625
2018-06-04T19:56:58.664880: step 495, loss 2.13057e-06, acc 0.625
2018-06-04T19:56:58.762834: step 496, loss 1.5685e-06, acc 0.65625
2018-06-04T19:56:58.856399: step 497, loss 1.57852e-0

...

2018-06-04T20:22:59.299760: step 15736, loss -3.01685e-07, acc 0.328125
2018-06-04T20:22:59.439757: step 15737, loss 1.29652e-07, acc 0.90625
2018-06-04T20:22:59.560575: step 15738, loss 4.8354e-08, acc 0.5
2018-06-04T20:22:59.674824: step 15739, loss -1.47302e-07, acc 0.5
2018-06-04T20:22:59.793915: step 15740, loss 1.89057e-07, acc 0.546875
2018-06-04T20:22:59.920355: step 15741, loss -1.52491e-07, acc 0.546875
2018-06-04T20:23:00.082605: step 15742, loss 3.8737e-08, acc 0.765625
2018-06-04T20:23:00.198815: step 15743, loss 7.80209e-08, acc 0.75
2018-06-04T20:23:00.311113: step 15744, loss -1.36985e-07, acc 0.21875
2018-06-04T20:23:00.437292: step 15745, loss 1.71304e-07, acc 0.796875
2018-06-04T20:23:00.562058: step 15746, loss -1.01527e-07, acc 0.234375
2018-06-04T20:23:00.685573: step 15747, loss -1.25722e-09, acc 0.75
2018-06-04T20:23:00.807313: step 15748, loss 8.99719e-08, acc 0.703125
2018-06-04T20:23:00.938957: step 15749, loss -1.27078e-07, acc 0.5
2018-06-04T20:23:01.056843

2018-06-04T20:23:13.839879: step 15850, loss -1.88727e-05, acc 0.484375
2018-06-04T20:23:13.960748: step 15851, loss -1.00798e-05, acc 0.515625
2018-06-04T20:23:14.097437: step 15852, loss 3.36427e-05, acc 0.46875
2018-06-04T20:23:14.239428: step 15853, loss -3.35776e-05, acc 0.5
2018-06-04T20:23:14.350553: step 15854, loss 1.02657e-05, acc 0.234375
2018-06-04T20:23:14.475090: step 15855, loss 2.61717e-06, acc 0.390625
2018-06-04T20:23:14.615698: step 15856, loss -2.97771e-05, acc 0.65625
2018-06-04T20:23:14.754493: step 15857, loss 2.36842e-05, acc 0.3125
2018-06-04T20:23:14.880081: step 15858, loss -6.67182e-06, acc 0.9375
2018-06-04T20:23:14.994423: step 15859, loss -1.19989e-05, acc 0.5625
2018-06-04T20:23:15.115417: step 15860, loss 2.75074e-05, acc 0.4375
2018-06-04T20:23:15.233798: step 15861, loss -1.81349e-05, acc 0.625
2018-06-04T20:23:15.397478: step 15862, loss 2.57574e-06, acc 0.40625
2018-06-04T20:23:15.523196: step 15863, loss 1.75123e-05, acc 0.34375
2018-06-04T20:23:15

2018-06-04T20:23:28.851994: step 15965, loss -0.000156607, acc 0.734375
2018-06-04T20:23:28.962299: step 15966, loss 0.000253988, acc 0.28125
2018-06-04T20:23:29.091284: step 15967, loss -8.75829e-05, acc 0.703125
2018-06-04T20:23:29.237653: step 15968, loss -7.10934e-05, acc 0.6875
2018-06-04T20:23:29.380716: step 15969, loss 0.000164153, acc 0.234375
2018-06-04T20:23:29.532377: step 15970, loss -0.000148338, acc 0.546875
2018-06-04T20:23:29.669129: step 15971, loss 4.66e-05, acc 0.375
2018-06-04T20:23:29.799476: step 15972, loss 5.5693e-05, acc 0.3125
2018-06-04T20:23:29.961935: step 15973, loss -0.000113267, acc 0.6875
2018-06-04T20:23:30.115301: step 15974, loss 8.05537e-05, acc 0.5
2018-06-04T20:23:30.250893: step 15975, loss -4.19076e-05, acc 0.65625
2018-06-04T20:23:30.392799: step 15976, loss -8.46237e-05, acc 0.875
2018-06-04T20:23:30.511660: step 15977, loss 8.95404e-05, acc 0.328125
2018-06-04T20:23:30.639621: step 15978, loss -0.000114166, acc 0.53125
2018-06-04T20:23:30.76

2018-06-04T20:23:43.963792: step 16080, loss -7.67468e-05, acc 0.203125
2018-06-04T20:23:44.088902: step 16081, loss 6.00228e-05, acc 0.59375
2018-06-04T20:23:44.221309: step 16082, loss -1.10174e-05, acc 0.828125
2018-06-04T20:23:44.337177: step 16083, loss -3.78816e-05, acc 0.015625
2018-06-04T20:23:44.465376: step 16084, loss 6.82794e-05, acc 0.953125
2018-06-04T20:23:44.591699: step 16085, loss -4.46783e-05, acc 0.421875
2018-06-04T20:23:44.716886: step 16086, loss 1.93306e-05, acc 0.125
2018-06-04T20:23:44.838539: step 16087, loss 2.95912e-05, acc 0.953125
2018-06-04T20:23:44.960210: step 16088, loss -3.63347e-05, acc 0.015625
2018-06-04T20:23:45.092543: step 16089, loss 5.1882e-05, acc 0.703125
2018-06-04T20:23:45.210114: step 16090, loss -1.78158e-05, acc 0.78125
2018-06-04T20:23:45.334337: step 16091, loss -1.842e-05, acc 0.15625
2018-06-04T20:23:45.459111: step 16092, loss 3.51668e-05, acc 1
2018-06-04T20:23:45.581982: step 16093, loss -3.63053e-05, acc 0.234375
2018-06-04T20:

2018-06-04T20:24:01.011778: step 16195, loss 1.75938e-07, acc 0.921875
2018-06-04T20:24:01.145397: step 16196, loss 2.1478e-08, acc 0.078125
2018-06-04T20:24:01.262688: step 16197, loss -1.2704e-07, acc 0.4375
2018-06-04T20:24:01.338759: step 16198, loss 1.65113e-07, acc 0.6875
2018-06-04T20:24:01.428622: step 16199, loss -1.58757e-07, acc 0.046875
2018-06-04T20:24:01.559584: step 16200, loss 2.55282e-08, acc 0.875

Evaluation:
2018-06-04T20:24:01.779319: step 16200, loss 7.69764e-08, acc 0.96

Saved model checkpoint to /Users/justin/Desktop/Justin/2018-1/Capstone/Code/runs/1528109774/checkpoints/model-16200

2018-06-04T20:24:01.950304: step 16201, loss 6.14105e-08, acc 0.09375
2018-06-04T20:24:02.060320: step 16202, loss -1.23243e-07, acc 0.359375
2018-06-04T20:24:02.179500: step 16203, loss 1.57502e-07, acc 0.65625
2018-06-04T20:24:02.304592: step 16204, loss -9.83956e-08, acc 0.0625
2018-06-04T20:24:02.420517: step 16205, loss -1.01813e-08, acc 0.953125
2018-06-04T20:24:02.545486: s